# Template für Random-Forest Klassifizierung und regression | Single-Node/Single-GPU

***

In diesem Abschnitt wird erkläre, wie man den KMeans Algorithmus auf Worker skaliert und zum Laufen bringt. Dazu nutzen wir Dask und Rapids. Dask bietet uns ein Dashboard wo wir Einzelheiten verfolgen können. Dask-Cuda lässt sich gut mit PyTorch und anderen Frameworks kombinieren, ohne die Clustereinstellungen groß zu ändern.

In dem Nootebook für das Setup wird erklärt, wie man selber sowas auf seinem eigenen PC/Workstation/Cluster installiert. Es gibt auch andere Möglichkeiten das zu installieren. Hier nutzen wir ein Cluster Setup mit Nodes, installiert mittels Docker.   

<u>Hinweis:</u><br>
Wenn es mal hängt, muss ggf. der Cluster, Worker oder das Notebook neu gestartet werden. Sowas passiert, wenn z.B. eine Exception vorkommt.   

<br>

Dask: https://www.dask.org | https://docs.dask.org/en/stable/ <br>
Rapids: https://rapids.ai <br> 

# 1. Rapids und Dask

<img src="./pictures/raspids_oss.PNG"  width="825px;" hight="725px;">
(Bild: (https://on-demand.gputechconf.com/gtcdc/2018/pdf/dc8256-rapids-the-platform-inside-and-out.pdf))

Rapids bietet uns eine Sammlung von Libraries, die uns bei vielen Dingen unterstützen kann. Eine dieser Libraries, die wir nutzen werden, nennt sich cuDF. cuDF sind Cuda Dataframes, die erstmal nur auf einer GPU laufen. Die Idee dahinter ist, dass wir <u>nah zu dasselbe machen</u> wie bei Pandas (Panda DataFrame), aber nur auf einer GPU. Dadurch können viele Berechnungen viel schneller durchgeführt werden. Beispielsweise das Vorverarbeiten von Daten vor dem Trainieren, oder anderweitige Zwecke. Die API ist nah zu identisch wie bei Pandas. 

Rapids funktioniert <u>nur</u> mit <u>Nvidia Grafikkarten</u>.

Rapids ist OpenSource, die API Beschreibungen der Libraries verlinken den dahinterliegenden Code. Der Code enthählt selber nochmal Kommentare und erklärungen.

Rapids entwickelt sich und es kommen Updates sowie Erweiterungen. Algorithmen, die noch nicht (Single-/Multi-Node/ GPU) verfügbar sind, können noch in den neueren Versionen auftauchen. Das gilt besonders für Multi-Node/Multi-GPU Implementierungen der Algorithmen. Es kann sein das manche Multi-Node/Multi-GPU Algorithmen sich am Ende nicht zusammenfassen lassen können, oder das Speichern/Laden sowie die Verwendung dieser auf GPU/CPU eingeschränkt sind. Andere Multi-Node/Multi-GPU Algorithmen können so wie in diesem Template hier verwendet werden. Einge andere können sich in der Anwendung unterscheiden. 


Floating-Point Berechnungen mit cuDF werden auf der GPU parallel ausgeführt, also ist die Reihenfolge nicht deterministisch. Die Ergebnisse können von Pandas abweichen.
a + b ist nicht gleich b + a. Das kann z.B. bei data.sum() vorkommen.
cuDF unterstützt auch verschiedene Datentypen wie int32, int64, float32, float64. Die ganze Liste ist hier zu finden.

Es gibt noch Dask-cuDF, diese nutzt man unter anderem, um große Daten zu Partitionieren. Die API ist Dask-ähnlich. 

Eine weitere Library von Rapids ist cuML, welche auch cuDF nutzt. Diese enthält verschiedene supervised und unsupervsed Algorithmen. Der Fokus hier liegt dabei, dass die Algorithmen möglichst auf einer GPU laufen. Ein Teil dieser Algorithmen kann mittels eines Clusters (cuML nutzt dafür ein Dask Cluster) auf verschiedene Nodes und GPUs skalieren. 

Die API von cuML ist fast genauso aufgebaut wie bei 
Sklearn. <br> Die Release Notes von cuML sind hier zu finden (https://github.com/rapidsai/cuml/releases).
  

Was wir neben Rapids und Dask noch nutzen können ist CuPy [5]. CuPy ist wie Numpy, nur läuft es unter auf GPU. CuPy wird hier nicht detailliert erklärt.

<br> 

cuDF Cuda DataFrames Dok.:  https://docs.rapids.ai/api/cudf/stable/ <br>
cuDF API: https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.dataframe/ <br>
cuDF Floating-Point Berechnung: https://docs.rapids.ai/api/cudf/stable/user_guide/pandas-comparison/#floating-point-computation        <br>
cuDF Datentypen: https://docs.rapids.ai/api/cudf/stable/user_guide/data-types/ | https://docs.rapids.ai/api/cudf/stable/user_guide/io/ <br>

cuML GitHub: https://github.com/rapidsai/cuml  <br>
cuML Dok.: https://docs.rapids.ai/api/cuml/stable/ <br>

Dask-cuDF: https://docs.rapids.ai/api/dask-cudf/stable/

CuPy: https://cupy.dev



Was wir nutzen:<br>
cuDF: DataFrames deren API nahezu identisch mit Pandas ist. Also quasi Pandas auf GPU.<br>
dask_cudf: Dask bietet uns DataFrames, die verteilt sind. Damit kann man auch Daten partitionieren [11] <br>
cupy: wie Numpy und Scipy aber auf GPU.

<img src="./pictures/df.PNG"  width="725px;" hight="725px;">

Wir teilen das Trainieren in 4 Bereiche auf. Hier nutzen wir SNSG.

Das Bild zeigt in welchen Bereichen man welche DataFrames nutzen kann. Pandas liegt dabei nicht im Fokus.

<img src="./pictures/zuordnung.PNG"  width="1050px;" hight="1050px;">

Da wir mit Dask-cuDF und Dask große Daten lesen und partitionieren können, ist auch der Einsatz im SNSG Bereich möglich. cuDF unterstützt das lesen mehrere Dateien (bis jetzt)  nicht, so das man Dask oder Dask-cuDF nutzen muss, wenn man globstings nutzt, um alle CVS Dateien in einem Verzeichnis zu lesen.

Es gibt auch die Möglichkeit cuDF als Backend für Dask zu nutzen.

<br>

cuDF Backend für Dask: https://docs.rapids.ai/api/dask-cudf/stable/#dataframe-creation-from-on-disk-formats

<b>[Bild Konvertierungen]</b>


<u>Hinweis:</u><br>
Variablen die GPU Speicher nutzen und nicht mehr gebraucht werden, sollten mittels `del` gelöscht werden. Wenn nach dem Löschen immer noch Speicher belegt ist, muss der Kernel des Notebooks neugestartet werden. 

In [11]:
# Imports 
import cupy as   cp
import cudf
import cuml
from cuml.dask.common import utils as dask_utils
import dask_cudf
import dask

import numpy as np
from numpy import genfromtxt

import pandas as pd
import os
import time

Da das Interface wie Pandas aufgebaut ist, wird cuDF im detail nicht erklärt. Bei interesse kann man sich die API durchlesen.

In [3]:
# cuDF und Panda series 
s_cuda = cudf.Series([1, 2, cudf.NA])
s_panda = pd.Series([1, 2, pd.NA])

print(f"Pandas:\n{s_cuda}\n")
print(f"Rapids:\n{s_panda}")

Pandas:
0       1
1       2
2    <NA>
dtype: int64

Rapids:
0       1
1       2
2    <NA>
dtype: object


## 1.1 Start Dask-Cluster (Mit Docker) 

Pro GPU wird genau ein Prozess für einen Worker gestartet. Die Worker-ID wird hochgezählt. <br>

<u>Hinweise:</u><br>
Die MNMG Versionen der Algorithmen können auch auf SNSG laufen.

Wenn wir nur auf einen Knoten sind, können wir normal das Cluster-Setup nutzen oder einen lokalen Cluster erstellen. Hier nutzen wir den normalen Cluster mit einem GPU Worker.

<img src="./pictures/rapids_cluster.PNG">

In [1]:
import rapids_tools as rapids_tools   

Rapids-tools 1.0
Umgebungsvariablen werden gesetzt
NCCL_SOCKET_NTHREADS: 4
NCCL_NSOCKS_PERTHREAD: 2


In [2]:
# Erstelle Client 
client = rapids_tools.create_dask_client()  # Gebe IP an. Standard:  127.0.0.1:8786 (Scheduler ist da wo auch Jupyter läuft) 
client                                                           

Client IP: 127.0.0.1:8786


<Client: 'tcp://149.201.182.203:8786' processes=1 threads=1, memory=62.53 GiB>

Wenn wir eigene Python Module erstellen, können wir diese auch mit dem Cluster teilen.

In [3]:
# Lade Datei mit Dask hoch. Gibt Status aus... 
client.upload_file('rapids_tools.py')  # Nur als Beispiel

{'tcp://149.201.182.203:42963': {'status': 'OK'}}

In [4]:
# Zeige Schlüssel. Die Ports ändern sich. 
client.has_what().keys()

dict_keys(['tcp://149.201.182.203:42963'])

In [5]:
# Starte alle Worker neu
client.restart_workers(client.has_what().keys())
client.has_what().keys()

dict_keys(['tcp://149.201.182.203:39649'])

In [9]:
# Startet client neu.
#client.restart()

# 2. Daten

## 2.1 Daten generieren

Als Anfang erstellen wir Daten. Daten die auf dem Host sind, müssen erst verteilt werden.

cuML bietet uns Möglichkeiten Datensets zu erstellen.

In [6]:
# Imports 
from cuml.datasets  import make_blobs      # Klassifikation    
from cuml.datasets  import make_regression # Regression

from cuml import datasets  
from cuml.model_selection import train_test_split

Ein Weg an Daten zu kommen, ist Datensets zu nutzen, die schon mit den Frameworks kommen. 

Das können wir nutzen, um den Aufbau des Codes zu testen und zu überprüfen, ob die Installation von Rapids und die Dask-Worker funktionieren.

Es gibt einige Parameter, die wir eingeben können, um Daten so zu generieren wie wir es wollen. Um es einfach zu halten, nutzen wir nur 2 Parameter.

<br>

Generiere Single-GPU Daten: https://docs.rapids.ai/api/cuml/stable/api/#dataset-generation-single-gpu

In [24]:
# Trainings- und Testgröße
train_size = 20000 # 20000
test_size = 100
n_samples = train_size + test_size

# Für Klassifizierung
X_c, y_c = datasets.make_blobs(n_samples=n_samples)
# Teile Daten auf                 
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split( X_c, y_c, train_size=0.80, random_state=5)

# Für Regression 
X_r, y_r = datasets.make_regression(n_samples=n_samples)
# Teile Daten auf                          
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split( X_r, y_r, train_size=0.80, random_state=5)


Jetzt können wir uns anschauen was gemacht wurde, so könnten unsere Trainingsdaten auch aussehen.

In [8]:
print(f"Daten Klassifizierung:\nX: {type(X_c)} y: {type(y_c)} \n X[0]: {X_c[0]}  \n y[0]: {y_c[0]} \n Länge X, y: {len(X_c)}, {len(y_c)}")

Daten Klassifizierung:
X: <class 'cupy.ndarray'> y: <class 'cupy.ndarray'> 
 X[0]: [-3.3717754 -8.665239 ]  
 y[0]: 0.0 
 Länge X, y: 20100, 20100


In [9]:
print(f"Daten Regression:\nX: {type(X_r)} y: {type(y_r)} \n X[0]: {X_r[0]}  \n y[0]: {y_r[0]} \n Länge X, y: {len(X_r)}, {len(y_r)}")

Daten Regression:
X: <class 'cupy.ndarray'> y: <class 'cupy.ndarray'> 
 X[0]: [-1.0874884  0.8080687]  
 y[0]: [-41.24021] 
 Länge X, y: 20100, 20100


## 2.3 Daten sind schon Lokal verfügbar

Mit cuDF können wir Daten direkt in die GPU laden und Daten schreiben. Es gibt viele Parameter, die angegeben werden können. 

Wenn die Daten nicht in den Speicher passen, können diese partitioniert werden, indem wir die Daten mit Dask-cuDF laden.

<img src="./pictures/rapids_chunk.PNG">

Wir können Daten mit Dask laden und daraus ein Dask-cuDF erstellen.

Intern nutzt jeder Worker cuDF um die Daten zu lesen. Es können verschiedene Quellen angegeben werden wie HDFs, S3, http, ... (https://docs.dask.org/en/stable/how-to/connect-to-remote-data.html)

<br>

cuDF Input/Output: https://docs.rapids.ai/api/cudf/legacy/api_docs/io/#input-output <br>
Dask-cuDF Creating and storing DataFrames: https://docs.rapids.ai/api/dask-cudf/stable/api/#creating-and-storing-dataframes

In [6]:
df = cudf.DataFrame(
    {
        "a": list(range(20)),
        "b": list(reversed(range(20))),
        "c": list(range(20)),
    })

In dem Fall, dass die Daten sehr groß sind oder wir viele Daten lesen wollen, können wir alternativ zu cuDF auch Dask-cuDF nutzen. Je nach Version unterstützt cuDF noch nicht das Lesen mehrere Dateien.

Die Anwendung von Dask-cuDF ist einfach. Wer Dask schon kennt, die API von Dask-cuDF ist nahzu identisch mit Dask.

Beim lesen können wir eine Blocksize angeben. Dann wird eine Anzahl an Partitionen erstellt. 

Für ein kurzes Beispiel, schreiben wir eine Datei und lesen diese mit Dask-cuDF.

In [58]:
# save data
np.savetxt("csv_file.csv", cp.asnumpy(X_train_c), delimiter=",", header="A,B")


In [59]:
# "*.csv" um alle CSV Dateien zu lesen
dcf = dask_cudf.read_csv("csv_file.csv", blocksize="100KB")  
dcf

,# A,B
npartitions=9,,
,float64,float64
,...,...
...,...,...
,...,...
,...,...


In [60]:
type(dcf)

dask_cudf.core.DataFrame

Für unseren Versuch haben wir die Trainingsdaten X. Beim lesen mit dask_cudf versucht der Worker dann den gegebenen Pfad zu lesen. Wir arbeiten ggf. in einem anderen Verzeichnis.
Damit der Worker die Datei sieht, muss der Worker diese schreiben.

In [61]:
@dask.delayed
def write_data(X):
    import cupy as cp
    np.savetxt("csv_file.csv", cp.asnumpy(X), delimiter=",", header="A,B")
fut = write_data(X_train_c)
fut.compute()

Jetzt hat dieser Worker die Datei und wir können die Datei lesen.

Mit Persist sagen wir, das die Daten aktiv im Speicher gehalten werden.

In [62]:
dcf = dask_cudf.read_csv("csv_file.csv", blocksize="100KB")  

dcf = dcf.persist()
dcf

,# A,B
npartitions=9,,
,float64,float64
,...,...
...,...,...
,...,...
,...,...


In [29]:
type(dcf)

dask_cudf.core.DataFrame

X und y müssen die selbe Anzahl an Partitionen haben. Da wir y noch nicht partitioniert haben, müssen wir das machen.

In [70]:
partitionen = 9
dcf_y = dask_cudf.from_cudf( cudf.Series(y_train_c), npartitions=partitionen )
dcf_y = dcf_y.persist()
dcf_y

<dask_cudf.Series | 9 tasks | 9 npartitions>

In [30]:
# Damit sehen wir die Patitionen
client.has_what()

{'tcp://149.201.182.203:39649': ("('read-csv-ed8861c034a65772f82ec2304d734821', 0)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 1)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 2)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 3)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 4)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 5)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 6)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 7)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 8)")}

### Trainieren

Mit den partitionierten Daten können wir jetzt das Model trainieren.

In [66]:
from cuml.ensemble import RandomForestClassifier as single_RandomForestClassifier  

In [69]:
%time single_RandomForestClassifier.fit(x=dcf, y=dcf_y)  

IndexError: tuple index out of range

Eine Alternative wäre, die Partitionen stückweise zu nutzen. 

In [94]:
%%time
# Alternativ: 

partitions = 6  # Anzahl partitionen
single_model = single_Kmeans(init="k-means||", random_state=100)

for i in range(partitions):
    # Gehe Partitionen durch.
    # - So können pro Iteration Daten ausgegeben werden. 
    single_model.fit(dcf.get_partition(i))
    

CPU times: user 75.3 ms, sys: 99 µs, total: 75.4 ms
Wall time: 142 ms


# 3. Training

Wie bereits erwähnt, ist die API fast genau so wie bei Sklearn. Wenn wir jetzt Daten haben und diese noch verarbeiten wollen, muss darauf geachtet werden, dass die Datentypen am Ende richtig sind.


Für das Model gibt es einige Parameter, die wir einstellen können, dieser Auszug zeigt alle möglichen Parameter <br>
Random-Forest Klassifikation:<br>
`split_criterion: =   (0 or 'gini' | 1 or 'entropy' ) ` <br>
`max_depthint= (default = 16) ` <br>
`max_leavesint= (default = -1)` <br>
`n_binsint= (default = 128)` <br>
`n_streams=4 Number of parallel streams used for forest building. ` <br>
Für Regression: <br>
`split_criterion: =   ( 2 or 'mse' | 4 or 'poisson' | 5 or 'gamma' | 6 or 'inverse_gaussian') `<br>
`accuracy_metric= ( r2 | median_ae | mean_ae | mse ) Decides the metric used to evaluate the performance of the model. ` <br>
`max_depth= (default = 16) ` <br>
`max_leaves= (default = -1)` <br>
`n_bins= (default = 128)` <br>
`n_streams=4 Number of parallel streams used for forest building. ` <br>

<br>

Sklearn Random-Forest: <br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html <br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html <br>
cuML SNSG Random-Forest: https://docs.rapids.ai/api/cuml/stable/api/#random-forest

In [71]:
from cuml.ensemble import RandomForestClassifier as single_RandomForestClassifier  
from cuml.ensemble import RandomForestRegressor  as single_RandomForestRegressor  

Jetzt kann man wie üblich vorgehen. Das Model Trainieren, Speichern, Laden und Evaluieren.

Wir können das Dashboard von Dask öffnen und sehen unter GPU die Auslastung (wenn es nicht zu schnell fertig ist).

In [72]:
# Klassifikation
single_rm_c = single_RandomForestClassifier()
                     
# Training  
%time single_rm_c.fit(X_train_c, y_train_c)

CPU times: user 199 ms, sys: 163 ms, total: 362 ms
Wall time: 186 ms


RandomForestClassifier()

In [75]:
# Regression
single_rm_r = single_RandomForestRegressor()
                     
# Training  
%time single_rm_r.fit(X_train_r, y_train_r)

CPU times: user 462 ms, sys: 364 ms, total: 825 ms
Wall time: 283 ms


RandomForestRegressor()

Nach dem Trainieren können wir Predictions machen.

In [76]:
# Klassifikation
to_predict = X_test_c
print(f"Länge: {len(to_predict)} \nTyp: {type(to_predict)}\n {to_predict}")

Länge: 4020 
Typ: <class 'cupy.ndarray'>
 [[-2.2514944   0.35604453]
 [-0.63367057  0.10339082]
 [-0.98335     1.2653158 ]
 ...
 [-0.45035046  9.211267  ]
 [-2.2621493   0.6378769 ]
 [-0.6701423  -0.18006289]]


In [77]:
# Klassifikation
single_rm_pred_c = single_rm_c.predict(to_predict)
# Als Output bekommen wir ein cupy Array
type(single_rm_pred_c)

cupy.ndarray

# 3. Speichern, Laden, Testen

Das Speichern und Laden des Modelsgeht einfach mit nur wenigen Schritten. Das Evaluieren ist ähnlich wie bei Sklearn.

In [78]:
import pickle
# Speichere Single-Node/Single-GPU Model 
# RM Klassifizierer
pickle.dump(single_rm_c, open("template_data_RandomForest/cuml_SingleNode_RM_c_model.pkl", "wb"))
# RM Regressor
pickle.dump(single_rm_r, open("template_data_RandomForest/cuml_SingleNode_RM_r_model.pkl", "wb"))
# Lade:
loaded_model_c = pickle.load(open("template_data_RandomForest/cuml_SingleNode_RM_c_model.pkl", "rb"))
loaded_model_r = pickle.load(open("template_data_RandomForest/cuml_SingleNode_RM_r_model.pkl", "rb"))

In [80]:
# Auch mit cuML möglich
#   cuml.metrics.accuracy_score()
#   cuml.metrics.r2_score()

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

Wir können Metriken anwenden.

Um Numpy zu nutzen, müssen wir nur  `.get()` hinter unsere Variablen schreiben, wie: `y.get()`. Oder wir wandeln die Daten vorher in ein Numpy um.

In [81]:
print(f"Type X_c: {type(X_c)}\nType X_c.get(): {type(X_c.get())}")

Type X_c: <class 'cupy.ndarray'>
Type X_c.get(): <class 'numpy.ndarray'>


In [82]:
# Eval RM Klassifizierer 
predictions_c = loaded_model_c.predict ( X_test_c )
score__c      = accuracy_score( y_test_c.get(), predictions_c.get() )


# Eval RM Regressor 
predictions_r = loaded_model_r.predict( X_test_r )
score__r = r2_score( y_test_r.get(), predictions_r.get() )

print(f"score__c: {score__c} \nscore__r: {score__r}")

score__c: 0.995273631840796 
score__r: 0.998159362737727


# 4. Treelite

Jetzt schauen wir uns den Fall an, dass das Zielsystem keine GPU hat. Wir trainieren das Model mit GPUs und nutzen es in einem anderen System, dass keine GPUs besitzt. Für solche fälle können wir Random-Forest in einen allgemeinen Baum umwandeln.

Auf dem Zielsystem muss dann nur die Library Treelite und numpy installiert sein.

Das cuML Model bietet Intern schon Möglichkeiten Treelite Attribute zu extrahieren. 

<br>

Treelite Dok.: https://treelite.readthedocs.io/en/latest/ <br>
Wandel cuMl RM in Treelite um: https://docs.rapids.ai/api/cuml/nightly/pickling_cuml_models/#Exporting-cuML-Random-Forest-models-for-inferencing-on-machines-without-GPUs

In [83]:
# Gebe ein Checkpoint Pfad an
checkpoint_path = './template_data_RM_Multi/treelite_single_c.tl'
# Klassifikation
loaded_model_c.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

# Gebe ein Checkpoint Pfad an
checkpoint_path = './template_data_RM_Multi/treelite_single_r.tl'
# Regression
loaded_model_r.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

Danach muss das Model auf das Zielsystem kopiert werden. Dort sollte die Library treelite installiert sein, z.B. mit `pip install treelite`. Rapids bzw. cuML muss nicht vorhanden sein.

In [84]:
# Auf dem Zielsystem #

# Treelite 
import treelite

# The checkpoint file has been copied over
checkpoint_path = './template_data_RM_Multi/treelite_single_c.tl'
tl_model_c = treelite.Model.deserialize(checkpoint_path)

# Regression
checkpoint_path = './template_data_RM_Multi/treelite_single_r.tl'
tl_model_r = treelite.Model.deserialize(checkpoint_path)

In [85]:
treelite.gtil.predict(tl_model_r, np.asarray([[1., 2.]],  dtype=np.float32))

array([[180.33533]], dtype=float32)

Wir können wieder die Genauigkeit testen.

In [86]:
predictions_c_treelite = treelite.gtil.predict(tl_model_c, np.asarray(X_test_c.get()))
score__c      = accuracy_score( y_test_c.get(), predictions_c_treelite )

predictions_r_treelite = treelite.gtil.predict(tl_model_r, np.asarray(X_test_r.get()))
score__r      = r2_score( y_test_r.get(), predictions_r_treelite )


print(f"score_c: {score__c} \nscore_r: {score__r}")

score_c: 0.995273631840796 
score_r: 0.9981593630258441


In [88]:
rapids_tools.arrays_almost_equal( cp.asnumpy( predictions_r ),  predictions_r_treelite )

Gebe 10 Zeilen aus, die ungleich sind
X: 99.97577667236328 	!= 	 y: 99.97579956054688 
X: 42.72260665893555 	!= 	 y: 42.72258758544922 
X: 91.18220520019531 	!= 	 y: 91.18222045898438 
X: -48.88114547729492 	!= 	 y: -48.88114929199219 
X: -109.8902816772461 	!= 	 y: -109.89027404785156 
X: -163.81092834472656 	!= 	 y: -163.81101989746094 
X: -74.39852142333984 	!= 	 y: -74.39851379394531 
X: -27.513090133666992 	!= 	 y: -27.513071060180664 
X: -88.20382690429688 	!= 	 y: -88.203857421875 
X: -30.911109924316406 	!= 	 y: -30.91110610961914 


AssertionError: 
Arrays are not equal

Mismatched elements: 3685 / 4020 (91.7%)
Max absolute difference: 0.00024414
Max relative difference: 1.765497e-06
 x: array([ 99.97578 ,  42.722607,  91.182205, ..., -30.84278 ,  79.23003 ,
       104.50507 ], dtype=float32)
 y: array([ 99.9758  ,  42.722588,  91.18222 , ..., -30.842789,  79.23004 ,
       104.50512 ], dtype=float32)

In [89]:
client.shutdown()